In [1]:
### https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/

In [2]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [3]:
# Helper Functions

def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# Constructs the Tokenizer for each language in the Translation
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# Determines the Longest Sentences contained in the Dataset
def max_length(lines):
    return max(len(line.split()) for line in lines)

# Encodes and Pad word sequences to fill the maximum length
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

In [4]:
# Filename before the train/test and filetype
filename = "phoenix"

dataset = load_clean_sentences(filename + '-both.pkl')
train = load_clean_sentences(filename + '-train.pkl')
test = load_clean_sentences(filename + '-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])

# prepare data
x_train = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
x_test = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

In [5]:
# Loading the model created in training
model = load_model('model3.h5')

In [6]:
# Evaluation Helper Functions

# Maps an Integer to each Word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = []
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = [], []
    for i, source in enumerate(sources):
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    
    # Calculating BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [7]:
print('train')
evaluate_model(model, eng_tokenizer, x_train, train)

train
src=[morgen wetter wie aussehen morgen mittwoch siebte oktober], target=[und nun die wettervorhersage far morgen mittwoch den siebten oktober], predicted=[und nun die wettervorhersage far morgen montag den achten oktober]
src=[wind wehen wind sturm nord sturm region orkan haben moeglich], target=[von dort sieht am nachmittag zum teil kraftiger regen heran oberhalb von vierhundert bis achthundert metern fallt schnee], predicted=[der wind weht bis bis bis bis bis frisch frisch frisch der der der sturmbaen]
src=[jetzt wie aussehen wetter morgen donnerstag dreizehn august], target=[und nun die wettervorhersage far morgen donnerstag den dreizehnten august], predicted=[und nun die wettervorhersage far morgen mittwoch den zwalften august]
src=[heute nacht schnee regen region suedost region auch schnee stark], target=[heute nacht regnet oder schneit es hier und da etwas im sadosten bayerns anfangs auch noch kraftiger], predicted=[heute nacht nacht es es im im im im im im im im und]
src=[

In [8]:
print('test')
evaluate_model(model, eng_tokenizer, x_test, test)

test
src=[dienstag nord fluss nebel ix sonne], target=[am dienstag im norden am oberrhein und an der donau oft zaher nebel sonst meist sonne], predicted=[am dienstag im im im und und und und und und]
src=[tag drei bis neun grad region], target=[am tag dann drei grad bis neun grad], predicted=[am tag minus grad grad und und und und grad grad]
src=[minus eins f insel minus zehn ost], target=[morgen minus ein grad auf den friesischen inseln minus zehn im osten], predicted=[am tag minus grad grad grad und minus minus plus grad grad]
src=[das ist es zeigen bildschirm montag anfang viel wechselhaft kuehl kalt], target=[die neue woche beginnt wechselhaft und winterlich kalt], predicted=[die den neuen tagen tagen in in und wetter wetter wetter]
src=[morgen schwach ix wehen ix wehen dann nord sturm], target=[morgen schwacher bis maaiger im norden frischer wind aus west bis sadwest an der see starmische baen], predicted=[morgen schwacher maaiger maaiger bis wind der der der bis bis bis bis]
src=